Importing necessary libraries and supporting files

In [1]:
import pandas as pd
import numpy as np
import xlwt
from xlwt import Workbook
from datetime import date
import math
import xlsxwriter
from pandas import read_excel


Available = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Available.csv')
Down = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Down.csv')
Leased = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Leased.csv')
Light5 = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Light5.csv')
PC = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/PC.csv')
Yardi = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Yardi.csv')
Procore = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Procore.csv')
PropertyLookUpTable = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/PropLookUp.csv')
# Residents = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Residents.csv')
Residents = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Residents.xlsx', sheet_name = 'Report1')

This function creates a column titled 'Combo' which contains <Property Number>_<Unit Number>
it also eliminates any leading 0's any '.0' which is sometimes found in the Property Number

In [2]:
def makeComboUnits(db):
    unitNum = db['Unit'].astype(str)
    ind = 0
    for val in unitNum:
        while True:
            if (val[0] == '0'):
                val = val[1:]
            else:
                unitNum[ind] = val
                ind = ind + 1
                break
                
    db['Combo'] = db['Property'].astype(str) + '_' +unitNum
    db['Combo'] = db['Combo'].str.replace('.0', '', regex=False)


Calling the function with the supporting data

In [3]:
CurrentResidents = Residents.loc[(Residents['Status'] == 'Current') | (Residents['Status'] == 'Future')]
PastResidents = Residents.loc[Residents['Status'] == 'Past']
CurrentResidents = pd.DataFrame.reset_index(CurrentResidents)
PastResidents = pd.DataFrame.reset_index(PastResidents)
PastResidents = PastResidents.sort_values(by = 'Move Out', ignore_index=True, ascending = False)


makeComboUnits(Yardi)
makeComboUnits(Available)
makeComboUnits(Down)
makeComboUnits(Leased)
makeComboUnits(Light5)
makeComboUnits(PC)
makeComboUnits(CurrentResidents)
makeComboUnits(PastResidents)

This is where the weird logic is located.  Instead of trying to determine the unit numbers from each Procore project, it is easier to simply break up each word in the Procore Project and create a combo unit with it.  For example if the job is 
'225 Mallorca 101, +205, (301), -307'
this will create combo string of 
['4301_225', '4301_Mallorca', '4301_101', '4301_+205', 4301_(301)', '4301_-307']
We will store this information into a dataframe called 'Procore_Units'

We can then see if any of our Down units are found in this list which is the end goal.

Some small workarounds we need to do with this approach:
    Ignore the first word in the Procore Project string, if there exists a 225 Mallorca unit 225 this will create issues.
    Remove any '+' ',' or ' ', that might create some bugs down the line.

In [4]:
Procore['Property Number'] = Procore['Project Number'].astype(str)

Procore_Units = pd.DataFrame(columns = ['Combo', 'Stage'])
for proj in Procore['Name']:
    terms = proj.split()
    index = 1
    while True:
        
        unitNum = terms[index]
        unitNum = str(unitNum)
        while True:
            if (unitNum != ''):
                if (unitNum[0] == '0'):
                    unitNum = unitNum[-1*(len(unitNum))+1:]
                else:
                    break
            else:
                break

        projNum = Procore.loc[Procore.Name == proj, 'Project Number'].values[0]
        comboStr = str(projNum) + '_'+ str(unitNum)
       
        deletedChar = [',', ' ', '+']
       
        for char in deletedChar:
            comboStr = comboStr.replace(char,'')
            
        stageStr = Procore.loc[Procore.Name == proj, 'Stage']
        Procore_Units = Procore_Units.append({'Combo': comboStr, 'Stage': stageStr }, ignore_index=True)        
        index = index + 1
        
        if (index == len(terms)):
            break
        
Procore_Units['Combo'] = Procore_Units['Combo'].str.replace('.0', '', regex=False)

In [5]:
Yardi['Status'] = np.nan
Yardi['Procore Status'] = np.nan
Yardi['Light5 Status'] = np.nan
Yardi['PC Status'] = np.nan
Yardi['Move In'] = np.nan
Yardi['Resident Name'] = np.nan
Yardi['Resident Number'] = np.nan
Yardi['Vacancy Date'] = np.nan

for unit in Yardi['Combo']:
    
    if (unit in Procore_Units['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Procore Status'] =  Procore_Units.loc[Procore_Units.Combo==unit,'Stage'].values[0].values[0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'Procore Status'] = 'N/A'

    
    if (unit in Light5['Combo'].values):
        if (Light5.loc[Light5.Combo == unit, 'Job Status'].values[0] == 'Completed'):
            Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = Light5.loc[Light5.Combo == unit, 'Job Status'].values[0] + ' - ' + str(Light5.loc[Light5.Combo == unit, 'Actual \nCompletion Date'].values[0])
        else:
            Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = Light5.loc[Light5.Combo == unit, 'Job Status'].values[0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = 'N/A'

                
    if (unit in PC['Combo'].values):
        PCHold = PC[PC['Combo']==unit]
        PCHold = PCHold.sort_values(by = 'Job Code', ignore_index=True, ascending=False)
        
        if (PCHold['Job Status'][0] == 'Completed'):
            Yardi.loc[Yardi.Combo == unit, 'PC Status'] = PCHold['Job Status'][0] + ' - ' + str(PCHold['Actual \nCompletion Date'][0])
        else:
            Yardi.loc[Yardi.Combo == unit, 'PC Status'] = PCHold['Job Status'][0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'PC Status'] = 'N/A'
 




    
    if (unit in Available['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Available'
        
    elif (unit in Down['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Down'
        
    elif (unit in Leased['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Leased'

        
    else:
        print(unit)
        
        
    if (unit in CurrentResidents['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Resident Movein'] =  str(CurrentResidents.loc[CurrentResidents.Combo==unit,'Move In'].values[0])[:10]
        Yardi.loc[Yardi.Combo == unit, 'Resident Name'] =  CurrentResidents.loc[CurrentResidents.Combo==unit,'Name'].values[0]
        Yardi.loc[Yardi.Combo == unit, 'Resident Number'] =  CurrentResidents.loc[CurrentResidents.Combo==unit,'Phone #s'].values[0]
        Yardi.loc[Yardi.Combo == unit, 'Vacancy Data'] = 'N/A'
        
    elif (unit in PastResidents['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Vacany Date'] =  str(PastResidents['Move Out'][PastResidents[PastResidents.Combo == unit].first_valid_index()])[:10]
        

    else:
        Yardi.loc[Yardi.Combo == unit, 'Resident Name'] = 'N/A'
        Yardi.loc[Yardi.Combo == unit, 'Resident Number'] = 'N/A'

In [6]:
class FitSheetWrapper(object):
    def __init__(self, sheet):
        self.sheet = sheet
        self.widths = dict()

    def write(self, r, c, label='', *args, **kwargs):
        self.sheet.write(r, c, label, *args, **kwargs)
        width = 5000
        self.sheet.col(c).width = width
#         if width > self.widths.get(c, 0):
#             self.widths[c] = width
#             self.sheet.col(c).width = width

    def __getattr__(self, attr):
        return getattr(self.sheet, attr)

Here we use the PropertyLookUpTable to get the property name for each project in Yardi.  This will help us later when creating the Grand Interface output file

In [7]:
PropertyLookUpTable['YARDI Code'].apply(lambda x: str(x))
propCodesDict = dict(zip(PropertyLookUpTable['YARDI Code'], PropertyLookUpTable['Cleaned Property']))
Yardi['Property Name'] = list(map(propCodesDict.get, Yardi['Property']))


This is where all of the printing to Excel happens now.  We have the status values and everything needed.  We pull values from the Property Look Up file to determine the PM, PA, and RM and then plot everything.

ListView

In [8]:
wb = Workbook()
columnTitles =['Property', 'Unit', 'Combo', 'Resident Name', 'Resident Number', 'Move In', 'Vacancy Date', 'Layout', 'Yardi Status', 'Procore Job','Light .5', 'Paint and Clean']

index = 0
length = 2774

while True:
    if (index == length):
        break
    propIndex = list(PropertyLookUpTable['YARDI Code']).index(int(Yardi['Property'][index]))
    propCombo = str(Yardi['Property'][index]) + ' - ' + str(Yardi['Property Name'][index])
    
    newSheet = FitSheetWrapper(wb.add_sheet('List View'))
  
    row = 0
    col = 0
    
    for info in columnTitles:
        newSheet.write(row,col,info)
        col = col + 1
    
    
    row = 1
    col = 0
    
    while True:
        prop = str(Yardi['Property Name'][index])
        unit = Yardi['Unit'][index]
        resName = Yardi['Resident Name'][index]
        resNum = Yardi['Resident Number'][index]
        resMove = Yardi['Resident Movein'][index]
        combo = Yardi['Combo'][index]
        layout = Yardi['Unit Type'][index][-3:]
        status = Yardi['Status'][index]
        prStatus = Yardi['Procore Status'][index]
        l5Status = Yardi['Light5 Status'][index]
        pcStatus = Yardi['PC Status'][index]
        vacDate = Yardi['Vacany Date'][index]
        
        columnValues = [prop, unit, combo, resName, resNum, resMove,vacDate, layout, status, prStatus, l5Status, pcStatus]
        col = 0
        
        for val in columnValues:
            val = str(val)

            
            newSheet.write(row, col, val)
            col = col + 1
        
        
        index = index + 1
        row = row + 1
        
        if(index == length):
            break

In [9]:
today = date.today()
fileName = ('LI ' + str(today) + '.xls')

wb.save(fileName)

Building View

In [10]:
Yardi = Yardi.sort_values(by = ['Property Name', 'Unit'], ignore_index=True)
wb = Workbook()
columnTitles =['Unit','Resident Name','Combo', 'Resident Number', 'Move In', 'Vacancy Date', 'Layout', 'Yardi Status', 'Procore Job','Light .5', 'Paint and Clean']

index = 0
length = 2774

while True:
    if (index == length):
        break
    prop = str(Yardi['Property'][index])
    propIndex = list(PropertyLookUpTable['YARDI Code']).index(int(Yardi['Property'][index]))
    
    propCombo = str(Yardi['Property'][index]) + ' - ' + str(Yardi['Property Name'][index])
    
    propMgr = 'PM: ' + PropertyLookUpTable['Prop Mgr'][propIndex]
    propAdmin = 'PA: ' + PropertyLookUpTable['Prop Admin'][propIndex]

    
    if (type(PropertyLookUpTable['Resident Manager'][propIndex]) == str):
        propRM = 'RM: ' + PropertyLookUpTable['Resident Manager'][propIndex]
    
    else:
        propRM = 'No RM'

    
    newSheet = FitSheetWrapper(wb.add_sheet(str(Yardi['Property Name'][index])))
  
    row = 0
    col = 0
    
    propInfo = [propCombo, propMgr, propAdmin, propRM]
    
    for info in propInfo:
        newSheet.write(row,col,info)
        col = col + 1
    
    
    row = 2
    col = 0
    
    for columnTitle in columnTitles:
        newSheet.write(row, col, columnTitle)
        col = col + 1
    
    row = 3
    col = 0
    
    
    
    while True:
        unit = 'Unit ' + Yardi['Unit'][index]
        resName = Yardi['Resident Name'][index]
        resNum = Yardi['Resident Number'][index]
        resMove = Yardi['Resident Movein'][index]
        combo = Yardi['Combo'][index]
        layout = Yardi['Unit Type'][index][-3:]
        status = Yardi['Status'][index]
        prStatus = Yardi['Procore Status'][index]
        l5Status = Yardi['Light5 Status'][index]
        pcStatus = Yardi['PC Status'][index]
        vacDate = Yardi['Vacany Date'][index]

        
        columnValues = [unit, resName, combo, resNum, resMove, vacDate, layout, status, prStatus, l5Status, pcStatus]
        col = 0
        for val in columnValues:
            val = str(val)

            
            newSheet.write(row, col, val)
            col = col + 1
        
        
        index = index + 1
        row = row + 1
        
        if(index == length):
            break
        if(prop != str(Yardi['Property'][index])):
            break

In [11]:
today = date.today()
fileName = ('GI ' + str(today) + '.xls')

wb.save(fileName)